In [3]:
!apt update
!apt install chromium-chromedriver -y
!pip install selenium webdriver-manager beautifulsoup4 pandas


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [344 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,361 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,287 kB]
Get:14 

In [5]:
# --- IMPORT ---
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# --- KONFIGURASI ---
VIDEO_URL = "https://youtu.be/YbTBIIoAz74?si=L-vgXeetsFi0GbyQ"
OUTPUT_CSV = "youtube_comments.csv"
MAX_SCROLLS = 100        # jumlah scroll (naikkan untuk komentar lebih banyak)
SCROLL_PAUSE = 2        # jeda antar scroll (detik)

# --- SETUP CHROME UNTUK COLAB ---
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)
driver.get(VIDEO_URL)
time.sleep(5)

# scroll beberapa kali supaya komentar muncul
body = driver.find_element(By.TAG_NAME, "body")
for i in range(5):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)

# scroll terus sampai habis atau mencapai batas
last_height = driver.execute_script("return document.documentElement.scrollHeight")
for i in range(MAX_SCROLLS):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(SCROLL_PAUSE)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        print("Selesai scroll di iterasi ke-", i+1)
        break
    last_height = new_height
    print(f"Scroll ke-{i+1}")

# ambil source HTML
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# parsing komentar
comments = []
for thread in soup.select("ytd-comment-thread-renderer"):
    text_elem = thread.select_one("#content-text")
    author_elem = thread.select_one("#author-text")
    time_elem = thread.select_one("#published-time-text")
    like_elem = thread.select_one("#vote-count-middle")

    if text_elem:
        comments.append({
            "author": author_elem.get_text(strip=True) if author_elem else "",
            "text": text_elem.get_text(strip=True),
            "publishedAt": time_elem.get_text(strip=True) if time_elem else "",
            "likeCount": like_elem.get_text(strip=True) if like_elem else ""
        })

# simpan ke CSV
df = pd.DataFrame(comments)
df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Disimpan {len(df)} komentar ke file {OUTPUT_CSV}")

# tampilkan 5 komentar pertama
df.head()


Scroll ke-1
Scroll ke-2
Scroll ke-3
Scroll ke-4
Scroll ke-5
Scroll ke-6
Scroll ke-7
Scroll ke-8
Scroll ke-9
Scroll ke-10
Scroll ke-11
Scroll ke-12
Scroll ke-13
Scroll ke-14
Scroll ke-15
Scroll ke-16
Scroll ke-17
Scroll ke-18
Scroll ke-19
Scroll ke-20
Scroll ke-21
Scroll ke-22
Scroll ke-23
Scroll ke-24
Scroll ke-25
Scroll ke-26
Scroll ke-27
Scroll ke-28
Scroll ke-29
Scroll ke-30
Scroll ke-31
Scroll ke-32
Scroll ke-33
Scroll ke-34
Scroll ke-35
Scroll ke-36
Scroll ke-37
Scroll ke-38
Scroll ke-39
Scroll ke-40
Scroll ke-41
Scroll ke-42
Scroll ke-43
Scroll ke-44
Scroll ke-45
Scroll ke-46
Scroll ke-47
Scroll ke-48
Scroll ke-49
Scroll ke-50
Scroll ke-51
Scroll ke-52
Scroll ke-53
Scroll ke-54
Scroll ke-55
Scroll ke-56
Scroll ke-57
Scroll ke-58
Scroll ke-59
Scroll ke-60
Scroll ke-61
Scroll ke-62
Scroll ke-63
Scroll ke-64
Scroll ke-65
Scroll ke-66
Scroll ke-67
Scroll ke-68
Scroll ke-69
Scroll ke-70
Scroll ke-71
Scroll ke-72
Scroll ke-73
Scroll ke-74
Scroll ke-75
Scroll ke-76
Scroll ke-77
Scroll k

,author,text,publishedAt,likeCount
0,@AyiFahmi460,Udah ga inget kapan terakhir nonton ini podkes...,5 months ago,2.6K
1,@ferryirwandi,podcast deg-degan ini,5 months ago,5.6K
2,@gazadansa813,Baru nonton ini karena Ferry irwandi ikut dem...,1 month ago,35
3,@diahayumoerti9758,Salut bang fery... buat sekolah yg bermanfaat ...,1 month ago,6
4,@rosnamendalo2198,Saya rasa ferry irwandi ini bukan musuh utama ...,5 months ago,786
